# Project 2 - Algorithmic methods for data mining
  # Retrieving data from Twitter
        
Cristina Menghini - StudentID: 1527821

The main task of the project is to build a graph that describese the network of users who tweetted about a certain specific query. To do that I achieved the following step:

# 1. Creation of APIs

I created an API that allowed me to retrieve data from twitter:

In [2]:
import tweepy
from tweepy import OAuthHandler
import time
import threading

In [3]:
cfg = { 
    "consumer_key"        : "FIdkHUbZuNjNxyPfImqI9TFXN",
    "consumer_secret"     : "lVyiiX67XiU3EkvsrC20yj2pDPhsQvUNFGr2FuGdvdmJxd1vwM",
    "access_token"        : "633193657-arMpLiTtLAlAGDfkUUqcPnx4obmgrE7VkTRUgJmO",
    "access_token_secret" : "2TLsxc9FUYUYVpeaTkPGkuL7Nmk5PwQQv5q9JbdrACZh8" 
}

auth = OAuthHandler(cfg["consumer_key"], cfg["consumer_secret"])
auth.set_access_token(cfg["access_token"], cfg["access_token_secret"]) 
api = tweepy.API(auth)

To speed up the retrieving operations I used many different APIs each of whose access key is stored into a file. That files that contain all the APIs keys are: 'consumer_key.txt', 'secret_consumer.txt', 'access_token.txt', 'secret_token.txt'. I decide to save the keys in external files because seeing the in the code makes it messy and, of course, also to not risk to lose them.

In [12]:
def ListKeys(doc_key):
    """This function creates the list of the keys of all APIs I'm going to use. 
    - doc_key: the file in which the keys are saved."""
    list_keys = [line for line in open(doc_key, 'r+')]
    keys = [list_keys[i].rstrip() for i in range(len(list_keys))]
    return keys

In [4]:
# Here I define the lists of the different keys. 
cons_key = ListKeys('consumer_key.txt')
cons_sec = ListKeys('secret_consumer.txt')
acc_tok = ListKeys('access_token.txt')
acc_tok_sec = ListKeys('secret_token.txt')
# Then I zip all of them in a unique variable.
input_crate_api = zip(cons_key, cons_sec, acc_tok, acc_tok_sec) # The len of that variable corresponds to the number of applications

In [6]:
def Create_API(var_input, a):
    """This function define a API using specific keys. Its return is the API.
    - var_input is the tuple of keys;
    - a is the app i refer to."""
    cfg = { 
    "consumer_key"        : var_input[a][0],
    "consumer_secret"     : var_input[a][1],
    "access_token"        : var_input[a][2],
    "access_token_secret" : var_input[a][3]
    }
    auth = OAuthHandler(cfg["consumer_key"], cfg["consumer_secret"])
    auth.set_access_token(cfg["access_token"], cfg["access_token_secret"]) 
    api = tweepy.API(auth)
    return api

# 2. Define the database 

Before starting the data download, I define a database in MongoDB ( based on 'localhost'). At first I built the db named 'Project' and then I am going to add collections step by step.

In [7]:
import pymongo
from pymongo import MongoClient
# Define the client of the db
client = MongoClient('localhost', 27017)
# Create the database
project_2 = client['Project']

At that point we are able to retrieve informations.

# 3. Download tweets

Download and collect tweets that contains a particular query. Suppose the topic is 'hello', each tweet will be saved as a dictionary:

                example_one_tweet = {'user_id' : '17','tweet' : 'hello world, Finally here I am!' }
    
The dictionaries will be filled in the specific 'user_tweet' collection of Project db. The user ids will be also stored in a file.

In [20]:
# Create the collection of users and their tweets
collection = project_2['Collection_Project']
us_tw = project_2.user_tweet # It's named 'user_tweet'

In [1]:
def Get_Users_and_Tweet(topic, num_tweet, max_tweet):
    """This function download the tweet and the id of the user who tweets, it also stores the data in user and tweet
    collection of MongoDB and gives back the list of users id.
    - num_tweet is the number of tweet to download per page;
    - max_tweet is the numer of total tweets that I want to collect."""
    tweet_count = 0  
    user_ids = [] # Inizialization of user_ids list
    while True:
        try:          
            for c in tweepy.Cursor(api.search, q = topic , count = num_tweet).pages(): # Scroll the cursor
                tweet_count += len(c) # Adjourn the number of collected tweets
                print tweet_count , 'tweets already collected.'
                us_tw.insert_many([{'user_id' : str(i.author.id) , 'tweet' : i.text} for i in c])# Insert elements in the collection
                user_ids += [i.author.id for i in c] 
                if tweet_count >= max_tweet : # Limit of tweets to download
                    break
        except tweepy.TweepError:
            print 'Requests excedeed. Sleep for 15 minutes.'
            time.sleep(15*60+10)
            continue # Continue to scroll the pages
        break 
    return user_ids

With the following lines of code I use the function define above to retrieve tweets and users. 

In [ ]:
ids_user = Get_Users_and_Tweet('yearofmercy', 100, 10000)
users = set(ids_user)

I decided to save some data also into files because it helps me to do more efficent operations( considerig the large scale of data) in the memory avoidig the research on the disk of the elements contained in local MongoDB.

In [ ]:
import pickle
pickle.dump( users, open('users.p', 'wb')) # Store the variable users in a file.

Up to now I collected the users and their tweets. By now I'm interesting in collecting the followers of each user. 

In order to study the network of users, I want to collect only the followers that are already in the list of users.

# 4. Collect followers

The procedure of retrieving followers consists in download data and store them in 'user_followers' collection MongoDB. Each element will be:

                        example_one_follower = {'user_id' : '17','follower' : '21'}
                        
This will be an entry of the collection and each of this element will be also saved into a file.

In [16]:
def Get_followers(us, API, thre):
    """This function downloads the followers of the users list and store them in Followers Mongodb collection.
    - us is the list of users id;
    - API is the api I use to download data;
    - num_page number of elements to consider in a page;
    - thre is the number of thread that's working."""  
    global list_followers
    for u in us:
        c = tweepy.Cursor(API.followers_ids, user_id = u, count = 5000).pages() # Define the cursor
        foll_user = []  # Define the list that will contain the lists of collected followers of user u for each page.        
        while True:                             
            try:
                foll_page = c.next() # Obtain the list of followers related to the first page
                foll_user += foll_page # Add the elements of the followers of that page to the list of the all followers of u  
            except tweepy.TweepError:
                print 'An error occurs. Sleep for 15 minutes. Thread:', thre
                e = threading.Event()
                e.wait(timeout = 15*60+10)
            except StopIteration: # When the list id u followers ends.
                break    
        followers = foll_user
        if len(followers) > 0 : # Check whether the list is empty, if it is not.
                fol_list = [{'user_id' : str(u), 'follower' : str(follower)} for follower in followers if str(follower) in users]# Create the list of edges btw users and followers that belong in the users list
                if len(fol_list) > 0: # To insert the list in Mongo, assure that it's length is grater than 0.
                    us_fol.insert_many(fol_list)#Here I add to Mongo collection the list of element each of which correspond to an edge of the graph.
                    list_followers += fol_list # Add elements to the list of all edges.
        print us_fol.count()
    print 'All followers collected.'

Since the large number of users that need to be processed I decide to use the multi-threading process that allows me to compute the operations in less time.

Therefore i split the list of users into n lists that correspond to the number of thread I'll run.

In [17]:
def users_chunks(n, users_list):
    """This function returns the list of chunks that will be used during the multi-threading.
    - n is the number of threads;
    - user_list is the list of user to split."""
    num = float(len(users_list))/n 
    us_lists = [ users_list [i:i + int(num)] for i in range(0, (n-1)*int(num), int(num))]
    us_lists.append(users_list[(n-1)*int(num):])
    return us_lists

Before running the multi-threading process I define the function that will be applied by each thread.

In [18]:
def worker(us_ID, Api , THREAD):
    """This function define the work that each thread is going to do.
    The variables of the function are the arguments of Get_followers function."""
    Get_followers(us_ID, Api, THREAD)

Hence, I define the new collection where will be stored the edges of the graph that will represent the networks.

In [19]:
# Create the collection 'user_followers'
us_fol = project_2.user_followers
us_fol.count()

145267

In [22]:
# Split the list of users into smaller lists
user_lists = users_chunks(26, list(users))

In [ ]:
#Initialize the entire list of edges
list_followers = []

In [ ]:
# Collect the followers
threads = []
for j in range(len(user_lists)):
    t = threading.Thread(target = worker, args = (user_lists[j], Create_API(input_crate_api, j), j)) # That's the funcion that each thread will execute
    threads.append(t)
    time.sleep(5)
    t.start()

For the next steps, to make the operations faster I save the edges also in a file.

In [ ]:
# Save the variable List_followers in a txt file
pickle.dump( list_followers, open( "save.p", "wb" ) )